# Individual Plot Deviation Reports (W36-W40)
This notebook processes each plot separately and generates individual Excel deviation reports.

## Workflow:
1. Mount Google Drive
2. Download large TIF from Google Drive
3. Crop into individual plots (W36-W40)
4. Load GeoJSON mask files from Google Drive
5. Process each plot with its corresponding mask GeoJSON
6. Generate individual deviation reports

In [ ]:
!pip install geopandas rasterio shapely openpyxl matplotlib pillow gdown

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
from PIL import Image as PILImage
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from matplotlib.ticker import ScalarFormatter
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
import os
import glob

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 2: Download and Crop Large TIF

In [ ]:
# Download the large TIF from Google Drive
!gdown 18bil2wZ48Zuch2WVdif-glvmQLKRplfi

Downloading...
From (original): https://drive.google.com/uc?id=18bil2wZ48Zuch2WVdif-glvmQLKRplfi
From (redirected): https://drive.google.com/uc?id=18bil2wZ48Zuch2WVdif-glvmQLKRplfi&confirm=t&uuid=fae6160d-0d08-47ac-94f1-b1897c9b900f
To: /content/W36-40_transparent_mosaic_group1.tif
100% 2.62G/2.62G [00:40<00:00, 64.8MB/s]


In [ ]:
# Crop the large TIF into individual plot TIFs
input_path = 'W36-40_transparent_mosaic_group1.tif'

# Coordinate bounds for each plot
coords = {
    36: {"TL": (232757.86, 282624.66), "BR": (232819.3, 281566.4)},
    37: {"TL": (232817.85, 282622.47), "BR": (232879.3, 281573.8)},
    38: {"TL": (232877.83, 282623.20), "BR": (232939.3, 281584.0)},
    39: {"TL": (232937.09, 282623.20), "BR": (232999.3, 281591.3)},
    40: {"TL": (232997.81, 282621.74), "BR": (233066.6, 281598.6)},
}

print("Cropping large TIF into individual plots...")

with rasterio.open(input_path) as src:
    for num, c in coords.items():
        left, top = c["TL"]
        right, bottom = c["BR"]

        # Create the window based on map coordinates
        window = from_bounds(left, bottom, right, top, src.transform)

        # Read the data inside the window
        data = src.read(window=window)

        # Update metadata for the cropped image
        out_meta = src.meta.copy()
        out_meta.update({
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": rasterio.windows.transform(window, src.transform)
        })

        out_path = f"W{num}.tif"
        with rasterio.open(out_path, "w", **out_meta) as dst:
            dst.write(data)

        print(f"✓ Wrote {out_path}")

print("\nAll plots cropped successfully!")

Cropping large TIF into individual plots...
✓ Wrote W36.tif
✓ Wrote W37.tif
✓ Wrote W38.tif
✓ Wrote W39.tif
✓ Wrote W40.tif

All plots cropped successfully!


## Step 3: Define Helper Functions

In [ ]:
# -----------------------------
# QC Points CSV Loader
# -----------------------------
def load_qc_points(qc_csv_path):
    """
    Load QC points CSV safely and return columns: y, x, z, point_id.
    Handles CK-style IDs and optional header row.
    """
    df = pd.read_csv(qc_csv_path, header=None, dtype=str)
    first_row = df.iloc[0, :4]
    if pd.to_numeric(first_row, errors='coerce').isnull().any():
        df = df.iloc[1:].reset_index(drop=True)
    else:
        df = df.reset_index(drop=True)

    df = df.iloc[:, :4].copy()

    df[0] = df[0].astype(str).str.replace(r'^\s*CK\s*', '', regex=True).str.strip()
    df[0] = pd.to_numeric(df[0], errors='coerce').astype("Int64")

    df[1] = pd.to_numeric(df[1], errors='coerce')
    df[2] = pd.to_numeric(df[2], errors='coerce')
    df[3] = pd.to_numeric(df[3], errors='coerce')

    out = pd.DataFrame({
        'y': df[1],
        'x': df[2],
        'z': df[3],
        'point_id': df[0]
    }).reset_index(drop=True)

    if out['point_id'].isnull().all():
        out['point_id'] = (out.index + 1).astype("Int64")

    return out

# -----------------------------
# Polygons
# -----------------------------
def load_polygons_geojson(geojson_path):
    return gpd.read_file(geojson_path)

def reproject_gdf(gdf, target_crs):
    return gdf.to_crs(target_crs)

# -----------------------------
# Geotiff
# -----------------------------
def get_geotiff_crs(tif_path):
    with rasterio.open(tif_path) as src:
        return src.crs

def get_geotiff_bounds(tif_path):
    """Get the geographic bounds of a GeoTIFF."""
    with rasterio.open(tif_path) as src:
        return src.bounds

def qc_points_to_gdf(qc_df, crs):
    return gpd.GeoDataFrame(
        qc_df,
        geometry=gpd.points_from_xy(qc_df['x'], qc_df['y']),
        crs=crs
    )

def filter_points_by_bounds(points_gdf, bounds):
    """Filter points that fall within the given bounds."""
    left, bottom, right, top = bounds
    return points_gdf[
        (points_gdf.geometry.x >= left) &
        (points_gdf.geometry.x <= right) &
        (points_gdf.geometry.y >= bottom) &
        (points_gdf.geometry.y <= top)
    ]

# -----------------------------
# Deviations
# -----------------------------
def compute_point_deviations(points_gdf, polygons_gdf):
    results = []
    for _, point in points_gdf.iterrows():
        point_id = point['point_id']
        containing = polygons_gdf[polygons_gdf.contains(point.geometry)]
        if not containing.empty:
            for _, poly in containing.iterrows():
                centroid = poly.geometry.centroid
                distance = point.geometry.distance(centroid)
                distance_cm = distance * 30.48
                results.append({
                    'point_id': point_id,
                    'polygon_id': poly.name,
                    'deviation_cm': distance_cm
                })
    return pd.DataFrame(results)

def export_polygon_centroids(polygons_gdf, output_csv_path):
    polygons_gdf['centroid'] = polygons_gdf.geometry.centroid
    centroids_df = polygons_gdf.copy()
    centroids_df['centroid_x'] = centroids_df['centroid'].x
    centroids_df['centroid_y'] = centroids_df['centroid'].y
    out = centroids_df[['centroid_x', 'centroid_y']]
    out.to_csv(output_csv_path, index=False)
    print(f"Centroids saved to {output_csv_path}")
    return out

# -----------------------------
# Excel report
# -----------------------------
def highlight_rows(row):
    return ['background-color: red' if row["deviation_cm"] > 3.0 else '' for _ in row]

def save_deviation_report(results_df, excel_path):
    if results_df.empty:
        print(f"⚠️ No deviations found. Creating empty report at {excel_path}")
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            results_df.to_excel(writer, index=False, sheet_name='Deviation Report')
        return

    styled = results_df.style.apply(highlight_rows, axis=1)
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        styled.to_excel(writer, index=False, sheet_name='Deviation Report')

    wb = load_workbook(excel_path)
    ws = wb['Deviation Report']
    last_row = ws.max_row + 2
    ws.cell(row=last_row, column=ws.max_column - 1, value="Average deviation (cm)")
    ws.cell(row=last_row, column=ws.max_column, value=results_df['deviation_cm'].mean())
    wb.save(excel_path)
    print(f"✓ Excel report saved: {excel_path}")

# -----------------------------
# GeoTIFF → JPEG
# -----------------------------
def geotiff_to_jpeg(tif_path, out_jpg_path):
    with rasterio.open(tif_path) as src:
        data = src.read().astype(np.uint8)
        data = np.transpose(data, (1, 2, 0))
        img = PILImage.fromarray(data)
        rgb_img = img.convert("RGB")
        rgb_img.save(out_jpg_path, "JPEG")

# -----------------------------
# Plotting
# -----------------------------
def plot_qc_and_centroids(tif_path, image_path, centroids_df, qc_df, output_path):
    """Plot orthophoto with centroids and QC points, save as PNG."""
    dataset = gdal.Open(tif_path)
    geotransform = dataset.GetGeoTransform()
    xmin = geotransform[0]
    xmax = xmin + geotransform[1] * dataset.RasterXSize
    ymax = geotransform[3]
    ymin = ymax + geotransform[5] * dataset.RasterYSize

    PILImage.MAX_IMAGE_PIXELS = None
    pil_img = PILImage.open(image_path)
    image = np.array(pil_img)
    if image.ndim == 2:
        image = np.stack([image]*3, axis=-1)

    def plot_points(ax, df, x_col, y_col, color, size, label, zorder):
        if not df.empty:
            ax.scatter(df[x_col], df[y_col], c=color, s=size,
                       label=label, edgecolor='black', zorder=zorder)

    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)
    ax.imshow(image, extent=[xmin, xmax, ymin, ymax])

    if not centroids_df.empty:
        plot_points(ax, centroids_df, 'centroid_x', 'centroid_y', 'green', 50, 'Detected Centroids', 1)

    if not qc_df.empty:
        qc_df['deviation_cm'] = qc_df.get('deviation_cm', 0)
        qc_white = qc_df[qc_df['deviation_cm'] <= 3]
        qc_red = qc_df[qc_df['deviation_cm'] > 3]

        plot_points(ax, qc_white, 'x', 'y', 'white', 20, 'QC Points', 2)
        plot_points(ax, qc_red, 'x', 'y', 'red', 40, 'Deviations > 3 cm', 3)

        for _, row in qc_red.iterrows():
            ax.annotate(str(int(row['point_id'])), (row['x'], row['y']),
                        textcoords="offset points", xytext=(0, 5),
                        ha='center', fontsize=10, color='white', zorder=4)

    ax.set_xlabel("Easting (ft.)")
    ax.set_ylabel("Northing (ft.)")
    ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0)
    ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    ax.ticklabel_format(useOffset=False, style='plain', axis='x')

    ticks = ax.get_xticks()
    ax.set_xticks([ticks[0], ticks[-1]])
    ax.set_xticklabels([f"{tick:.0f}" for tick in [ticks[0], ticks[-1]]], rotation=90)

    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Plot saved: {output_path}")

# -----------------------------
# Insert image into Excel
# -----------------------------
def insert_image_to_excel(excel_file, image_file, cell="E1", width=500):
    PILImage.MAX_IMAGE_PIXELS = None
    wb = load_workbook(excel_file)
    ws = wb.active
    with PILImage.open(image_file) as pil_img:
        orig_width, orig_height = pil_img.size
    new_height = int(width * (orig_height / orig_width))
    img = XLImage(image_file)
    img.width = width
    img.height = new_height
    ws.add_image(img, cell)
    wb.save(excel_file)

# -----------------------------
# Export centroids to WGS84
# -----------------------------
def export_centroids_wgs84(gdf, excel_file, sheet_name="Detected Centroids (WGS84)"):
    if gdf.empty:
        print("⚠️ No centroids to export to WGS84")
        return

    gdf_proj = gdf.to_crs(epsg=32610)
    gdf_proj['centroid'] = gdf_proj.geometry.centroid
    gdf_proj['centroid_wgs84'] = gdf_proj['centroid'].to_crs(epsg=4326)
    gdf_proj['X'] = gdf_proj['centroid_wgs84'].x
    gdf_proj['Y'] = gdf_proj['centroid_wgs84'].y
    centroids_df = gdf_proj[['X', 'Y']]

    wb = load_workbook(excel_file)
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
    else:
        ws = wb.create_sheet(sheet_name)

    # Write headers
    ws.cell(row=1, column=1, value="Y")
    ws.cell(row=1, column=2, value="X")
    ws.cell(row=1, column=3, value="ELEV")

    # Write data with ELEV column set to 166
    for row_idx, (_, row) in enumerate(centroids_df.iterrows(), start=2):
        ws.cell(row=row_idx, column=1, value=row['Y'])
        ws.cell(row=row_idx, column=2, value=row['X'])
        ws.cell(row=row_idx, column=3, value=166)

    wb.save(excel_file)
    print(f"✓ Centroids exported to WGS84 sheet in {excel_file} (with ELEV=166)")

## Step 4: Main Processing Function

In [ ]:
# -----------------------------
# Main Processing Function
# -----------------------------
def process_plot(plot_name, tif_path, geojson_path, qc_csv_path, output_dir="."):
    """
    Process a single plot and generate deviation report.

    Args:
        plot_name: Name of the plot (e.g., 'w36')
        tif_path: Path to the plot's GeoTIFF
        geojson_path: Path to the plot's mask GeoJSON
        qc_csv_path: Path to the QC points CSV (will be filtered to plot)
        output_dir: Directory to save outputs
    """
    print(f"\n{'='*60}")
    print(f"Processing {plot_name.upper()}")
    print(f"{'='*60}")

    # Define output paths
    excel_path = os.path.join(output_dir, f"{plot_name}_deviation_report.xlsx")
    ortho_jpg_path = os.path.join(output_dir, f"{plot_name}_ortho.jpg")
    qc_plot_path = os.path.join(output_dir, f"{plot_name}_qc_plot.png")
    centroids_csv_path = os.path.join(output_dir, f"{plot_name}_centroids_nad83.csv")

    # --- Load data ---
    print(f"Loading data for {plot_name.upper()}...")
    qc_df = load_qc_points(qc_csv_path)
    polygons_gdf = load_polygons_geojson(geojson_path)
    tif_crs = get_geotiff_crs(tif_path)
    tif_bounds = get_geotiff_bounds(tif_path)

    print(f"  TIF bounds: {tif_bounds}")

    # Reproject polygons to match TIF CRS
    polygons_gdf = reproject_gdf(polygons_gdf, tif_crs)

    # Convert QC points to GeoDataFrame
    points_gdf = qc_points_to_gdf(qc_df, tif_crs)

    # Filter QC points to only those within this plot's bounds
    points_gdf_filtered = filter_points_by_bounds(points_gdf, tif_bounds)
    qc_df_filtered = qc_df[qc_df['point_id'].isin(points_gdf_filtered['point_id'])]

    print(f"  Found {len(polygons_gdf)} polygons")
    print(f"  Found {len(points_gdf_filtered)} QC points within plot bounds (filtered from {len(qc_df)} total)")

    # --- Compute deviations ---
    results_df = compute_point_deviations(points_gdf_filtered, polygons_gdf)

    if not results_df.empty:
        print(f"  Computed {len(results_df)} deviations")
        print(f"  Average deviation: {results_df['deviation_cm'].mean():.2f} cm")
        print(f"  Max deviation: {results_df['deviation_cm'].max():.2f} cm")

        # Merge deviations into qc_df for plotting
        qc_df_filtered = qc_df_filtered.merge(
            results_df[['point_id', 'deviation_cm']],
            on='point_id',
            how='left'
        )
    else:
        print(f"  ⚠️ No deviations found for {plot_name.upper()}")

    # --- Export centroids ---
    centroids_df = export_polygon_centroids(polygons_gdf, centroids_csv_path)

    # --- Save deviation report ---
    save_deviation_report(results_df, excel_path)

    # --- Convert ortho to JPEG ---
    print(f"Converting {plot_name.upper()} ortho to JPEG...")
    geotiff_to_jpeg(tif_path, ortho_jpg_path)

    # --- Plot and save ---
    print(f"Creating visualization for {plot_name.upper()}...")
    plot_qc_and_centroids(tif_path, ortho_jpg_path, centroids_df, qc_df_filtered, qc_plot_path)

    # --- Insert plot into Excel ---
    insert_image_to_excel(excel_path, qc_plot_path, cell="E1", width=500)

    # --- Export centroids in WGS84 with ELEV column ---
    export_centroids_wgs84(polygons_gdf, excel_path)

    print(f"✓ {plot_name.upper()} processing complete!")
    print(f"  Report: {excel_path}")
    print(f"  Plot: {qc_plot_path}")

## Step 5: Configure Paths and Process All Plots

In [ ]:
!gdown 1pSLmuNU_OvgQMxbL4Q2a6S-VtiytHJXu

Downloading...
From: https://drive.google.com/uc?id=1pSLmuNU_OvgQMxbL4Q2a6S-VtiytHJXu
To: /content/W36-40 check points.csv
100% 5.98k/5.98k [00:00<00:00, 29.2MB/s]


In [ ]:
# -----------------------------
# Configure Paths
# -----------------------------

# Google Drive folder containing GeoJSON files
folder = "/content/drive/MyDrive/Orthos/W36-40 Deliverables/"

# QC points CSV (shared across all plots)
qc_csv_path = "W36-40 check points.csv"

# Find all GeoJSON files in the Google Drive folder
pattern = os.path.join(folder, "w*_mask_output.geojson")
geojson_files = glob.glob(pattern)

print(f"Found {len(geojson_files)} GeoJSON files:")
for f in geojson_files:
    print(f"  - {os.path.basename(f)}")

# Define plots to process based on found GeoJSON files
plots = {}
for geojson_path in geojson_files:
    # Extract plot name from filename (e.g., "w36_mask_output.geojson" -> "w36")
    filename = os.path.basename(geojson_path)
    plot_name = filename.split('_')[0]  # Gets "w36", "w37", etc.

    # Determine corresponding TIF file
    tif_name = plot_name.replace('w', 'W') + '.tif'  # "w36" -> "W36.tif"

    plots[plot_name] = {
        'tif': tif_name,
        'geojson': geojson_path
    }

print(f"\nConfigured {len(plots)} plots to process:")
for plot_name in sorted(plots.keys()):
    print(f"  {plot_name.upper()}: {plots[plot_name]['tif']} + {os.path.basename(plots[plot_name]['geojson'])}")

# Output directory (current directory by default)
output_dir = "."

Found 6 GeoJSON files:
  - w36_mask_output.geojson
  - w37_mask_output.geojson
  - w38_mask_output.geojson
  - w39_mask_output.geojson
  - w40_mask_output.geojson
  - w36-40_mask_output.geojson

Configured 6 plots to process:
  W36: W36.tif + w36_mask_output.geojson
  W36-40: W36-40.tif + w36-40_mask_output.geojson
  W37: W37.tif + w37_mask_output.geojson
  W38: W38.tif + w38_mask_output.geojson
  W39: W39.tif + w39_mask_output.geojson
  W40: W40.tif + w40_mask_output.geojson


In [ ]:
# -----------------------------
# Process All Plots
# -----------------------------

for plot_name, paths in plots.items():
    try:
        process_plot(
            plot_name=plot_name,
            tif_path=paths['tif'],
            geojson_path=paths['geojson'],
            qc_csv_path=qc_csv_path,
            output_dir=output_dir
        )
    except Exception as e:
        print(f"\n❌ Error processing {plot_name}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*60}")
print("All plots processed!")
print(f"{'='*60}")


Processing W36
Loading data for W36...
  TIF bounds: BoundingBox(left=232757.86, bottom=281566.40455, right=232819.30529999998, top=282624.66)
  Found 4556 polygons
  Found 33 QC points within plot bounds (filtered from 165 total)
  Computed 33 deviations
  Average deviation: 1.13 cm
  Max deviation: 2.33 cm
Centroids saved to ./w36_centroids_nad83.csv
✓ Excel report saved: ./w36_deviation_report.xlsx
Converting W36 ortho to JPEG...
Creating visualization for W36...


/usr/local/lib/python3.12/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


✓ Plot saved: ./w36_qc_plot.png
✓ Centroids exported to WGS84 sheet in ./w36_deviation_report.xlsx (with ELEV=166)
✓ W36 processing complete!
  Report: ./w36_deviation_report.xlsx
  Plot: ./w36_qc_plot.png

Processing W37
Loading data for W37...
  TIF bounds: BoundingBox(left=232817.85, bottom=281573.8007, right=232879.2953, top=282622.47000000003)
  Found 4518 polygons
  Found 33 QC points within plot bounds (filtered from 165 total)
  Computed 33 deviations
  Average deviation: 0.91 cm
  Max deviation: 2.30 cm
Centroids saved to ./w37_centroids_nad83.csv
✓ Excel report saved: ./w37_deviation_report.xlsx
Converting W37 ortho to JPEG...
Creating visualization for W37...
✓ Plot saved: ./w37_qc_plot.png
✓ Centroids exported to WGS84 sheet in ./w37_deviation_report.xlsx (with ELEV=166)
✓ W37 processing complete!
  Report: ./w37_deviation_report.xlsx
  Plot: ./w37_qc_plot.png

Processing W38
Loading data for W38...
  TIF bounds: BoundingBox(left=232877.82999999996, bottom=281584.0101000001

Traceback (most recent call last):
  File "rasterio/_base.pyx", line 310, in rasterio._base.DatasetBase.__init__
  File "rasterio/_base.pyx", line 221, in rasterio._base.open_dataset
  File "rasterio/_err.pyx", line 359, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: W36-40.tif: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipython-input-1193482963.py", line 7, in <cell line: 0>
    process_plot(
  File "/tmp/ipython-input-3065711326.py", line 29, in process_plot
    tif_crs = get_geotiff_crs(tif_path)
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-4071636835.py", line 50, in get_geotiff_crs
    with rasterio.open(tif_path) as src:
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/rasterio/env.py", line 463, in wrapper
    return f(*args, **kwds)
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-pack

## Step 6: Generate Summary Report (Optional)

In [ ]:
# -----------------------------
# Optional: Generate Summary Report
# -----------------------------

summary_data = []

for plot_name in plots.keys():
    excel_path = os.path.join(output_dir, f"{plot_name}_deviation_report.xlsx")

    if os.path.exists(excel_path):
        try:
            df = pd.read_excel(excel_path, sheet_name='Deviation Report')
            if not df.empty and 'deviation_cm' in df.columns:
                summary_data.append({
                    'Plot': plot_name.upper(),
                    'Total Points': len(df),
                    'Avg Deviation (cm)': df['deviation_cm'].mean(),
                    'Max Deviation (cm)': df['deviation_cm'].max(),
                    'Points > 3cm': len(df[df['deviation_cm'] > 3.0])
                })
        except Exception as e:
            print(f"Could not read {plot_name}: {e}")

if summary_data:
    summary_df = pd.DataFrame(summary_data)
    print("\nSummary of All Plots:")
    print(summary_df.to_string(index=False))

    # Save summary
    summary_path = os.path.join(output_dir, "all_plots_summary.xlsx")
    summary_df.to_excel(summary_path, index=False)
    print(f"\n✓ Summary saved to: {summary_path}")


Summary of All Plots:
Plot  Total Points  Avg Deviation (cm)  Max Deviation (cm)  Points > 3cm
 W36            35            1.128919            2.333082             0
 W37            35            0.909846            2.298767             0
 W38            35            0.997305            2.103352             0
 W39            35            1.184903            2.594675             0
 W40            35            1.373422            3.596471             1

✓ Summary saved to: ./all_plots_summary.xlsx
